In [9]:
# creating X & y from total and pheno
# import pandas as pd

# total_df = pd.read_csv('total.csv')
# pheno_df = pd.read_csv('pheno.csv')

# total_df = total_df.iloc[:, :6643]

# total_df = total_df.rename(columns={ total_df.columns[0]: 'SNP_ID' })
# pheno_df = pheno_df.rename(columns={ pheno_df.columns[0]: 'SNP_ID' })

# merged = pd.merge(total_df, pheno_df, on='SNP_ID', how='inner')

# X = merged[ total_df.columns.drop('SNP_ID') ]
# y = merged[ pheno_df.columns[-1] ]
# X.to_csv('X.csv', index=False)
# y.to_csv('y.csv', index=False)
# print(f"X shape: {X.shape}, y shape: {y.shape}")

X shape: (407, 6642), y shape: (407,)


In [13]:
###### rand1
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool
from joblib import Parallel, delayed
import seaborn as sns
import matplotlib.pyplot as plt

X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv').squeeze()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.3, random_state=1
)

baseline = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    random_seed=0,
    verbose=False,
    early_stopping_rounds=50
)
baseline.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
baseline_rmse = np.sqrt(mean_squared_error(y_val, baseline.predict(X_val)))
print(f"Baseline RMSE: {baseline_rmse:.4f}")

eps_schedule = [baseline_rmse * f for f in (1.05, 1.0, 0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.90)]

def sample_prior():
    return np.array([
        np.random.uniform(0.03, 0.15),  # learning_rate
        np.random.uniform(3, 10),       # depth
        np.random.uniform(0.01, 5)      # l2_leaf_reg
    ])

def simulate(theta):
    lr, depth, reg = theta
    model = CatBoostRegressor(
        learning_rate=lr,
        depth=int(round(depth)),
        l2_leaf_reg=reg,
        iterations=50,
        early_stopping_rounds=10,
        random_seed=0,
        verbose=False
    )
    model.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

def de_mutate(r1, r2, r3, F):
    return r1 + F * (r2 - r3)

def crossover(target, mutant, CR):
    trial = target.copy()
    dim = len(target)
    j_rand = np.random.randint(dim)
    for j in range(dim):
        if np.random.rand() < CR or j == j_rand:
            trial[j] = mutant[j]
    return trial

class ABC_SMC_DE:
    def __init__(self, N, eps_schedule, F=0.5, CR=0.9, strategy='rand/1', max_trials=100000):
        self.N = N
        self.eps_schedule = eps_schedule
        self.F = F
        self.CR = CR
        self.strategy = strategy
        self.max_trials = max_trials

    
    def run(self):
        particles, histories = [], []

        for t, eps in enumerate(self.eps_schedule):
            print(f"\n Population {t} (epsilon={eps:.3f})")
            prev_p = particles[-1] if t > 0 else None
            prev_s = histories[-1] if t > 0 else None

            accepted, sims = [], []
            trials = 0
            oversample = 5

            while len(accepted) < self.N and trials < self.max_trials:
                n_candidates = self.N * oversample
                proposals = []
                for _ in range(n_candidates):
                    theta = sample_prior() if prev_p is None else self.propose(theta_count=len(prev_p), prev_p=prev_p, prev_s=prev_s)
                    proposals.append(theta)

                # results = Parallel(n_jobs=-1)(

                results = Parallel(n_jobs=7)(

                    delayed(simulate)(theta) for theta in proposals

                )
                trials += len(results)

                for theta, sim_rmse in zip(proposals, results):
                    if sim_rmse <= eps:
                        accepted.append(theta)
                        sims.append(sim_rmse)
                        print(f"  {len(accepted)}. lr={theta[0]:.4f}, depth={int(round(theta[1]))}, reg={theta[2]:.4f}, RMSE={sim_rmse:.4f}")
                        if len(accepted) >= self.N:
                            break

                if len(accepted) == 0:
                    print(f"No acceptances after {trials} trials")
                    break

            if not accepted:
                raise RuntimeError(f"Population {t} failed: no accepts.")

            particles.append(np.vstack(accepted))
            histories.append(sims)
            print(f"Accepted {len(accepted)} in {trials} trials.")

        return particles, histories

    def propose(self, theta_count, prev_p, prev_s):
        N = theta_count
        if self.strategy == 'rand/1':
            i1, i2, i3 = np.random.choice(N, 3, replace=False)
            mutant = de_mutate(prev_p[i1], prev_p[i2], prev_p[i3], self.F)
        else:  # best/1
            best_idx = np.argmin(prev_s)
            r2, r3 = np.random.choice(N, 2, replace=False)
            mutant = de_mutate(prev_p[best_idx], prev_p[r2], prev_p[r3], self.F)

        target_idx = np.random.randint(N)
        theta = crossover(prev_p[target_idx], mutant, self.CR)
        theta[0] = np.clip(theta[0], 1e-4, 1.0)
        theta[1] = np.clip(theta[1], 2, 16)
        theta[2] = max(theta[2], 1e-6)
        return theta

if __name__ == '__main__':
    abc_de = ABC_SMC_DE(
        N=100,
        eps_schedule=eps_schedule,
        F=0.6,
        CR=0.9,
        strategy='rand/1',
        max_trials=200000
    )
    particles, histories = abc_de.run()

    final_s = histories[-1]
    best_idx = np.argmin(final_s)
    best_params = particles[-1][best_idx]
    best_lr, best_depth, best_reg = best_params

    print("\nBest hyperparameters:")
    print(f"  lr={best_lr:.4f}, depth={int(round(best_depth))}, reg={best_reg:.4f}")

    best_model = CatBoostRegressor(
        learning_rate=best_lr,
        depth=int(round(best_depth)),
        l2_leaf_reg=best_reg,
        iterations=500,
        random_seed=0,
        verbose=False
    )
    best_model.fit(Pool(X_train_full, y_train_full))
    preds = best_model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, preds))
    r2_test = r2_score(y_test, preds)
    print(f"Final Test RMSE: {rmse_test:.4f}, R²: {r2_test:.4f}")
    
    param_names = ['learning_rate', 'depth', 'l2_leaf_reg']
    for i, name in enumerate(param_names):
        plt.figure(figsize=(6, 2))
        for t, pop in enumerate(particles):
            sns.kdeplot(pop[:, i], label=f"Gen {t}")
        plt.title(f"Posterior evolution for {name}")
        plt.xlabel(name)
        plt.ylabel("Density")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    median_rmses = [np.median(s) for s in histories]
    min_rmses = [np.min(s) for s in histories]
    plt.figure(figsize=(7, 2))
    plt.plot(median_rmses, label="Median RMSE")
    plt.plot(min_rmses, label="Min RMSE")
    plt.xlabel("Population")
    plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
    plt.xticks(range(0,4))
    plt.ylabel("Validation RMSE")
    plt.title("ABC-SMC-DE: RMSE over Populations")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

Baseline RMSE: 48.4266

 Population 0 (epsilon=50.848)
  1. lr=0.0937, depth=6, reg=0.5745, RMSE=48.3144
  2. lr=0.0767, depth=9, reg=1.4214, RMSE=49.7094
  3. lr=0.1493, depth=4, reg=1.7439, RMSE=48.4329
  4. lr=0.1464, depth=5, reg=1.5201, RMSE=50.2256
  5. lr=0.1328, depth=5, reg=4.9255, RMSE=49.8980
  6. lr=0.0930, depth=9, reg=1.8018, RMSE=50.5702
  7. lr=0.0537, depth=4, reg=0.2363, RMSE=50.4172
  8. lr=0.1480, depth=8, reg=3.0516, RMSE=48.5190
  9. lr=0.0942, depth=3, reg=4.4908, RMSE=50.5243
  10. lr=0.1043, depth=3, reg=0.8448, RMSE=50.5566
  11. lr=0.0677, depth=5, reg=4.9046, RMSE=50.3655
  12. lr=0.0707, depth=10, reg=1.6591, RMSE=50.0822
  13. lr=0.0728, depth=4, reg=0.5161, RMSE=48.9381
  14. lr=0.0837, depth=5, reg=2.8842, RMSE=50.1753
  15. lr=0.1131, depth=6, reg=2.3502, RMSE=49.2386
  16. lr=0.1031, depth=6, reg=3.7739, RMSE=48.9012
  17. lr=0.1444, depth=7, reg=1.8091, RMSE=48.5245
  18. lr=0.0602, depth=4, reg=0.7644, RMSE=49.2434
  19. lr=0.0559, depth=7, reg=2.136

RuntimeError: Population 6 failed: no accepts.

In [9]:
###### best1 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool
from joblib import Parallel, delayed
import seaborn as sns
import matplotlib.pyplot as plt

X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv').squeeze()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.3, random_state=1
)

baseline = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    random_seed=0,
    verbose=False,
    early_stopping_rounds=50
)
baseline.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
baseline_rmse = np.sqrt(mean_squared_error(y_val, baseline.predict(X_val)))
print(f"Baseline RMSE: {baseline_rmse:.4f}")

eps_schedule = [baseline_rmse * f for f in (1.05, 1.0, 0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.90)]

def sample_prior():
    return np.array([
        np.random.uniform(0.03, 0.15),  # learning_rate
        np.random.uniform(3, 10),       # depth
        np.random.uniform(0.01, 5)      # l2_leaf_reg
    ])

def simulate(theta):
    lr, depth, reg = theta
    model = CatBoostRegressor(
        learning_rate=lr,
        depth=int(round(depth)),
        l2_leaf_reg=reg,
        iterations=50,
        early_stopping_rounds=10,
        random_seed=0,
        verbose=False
    )
    model.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

def de_mutate(r1, r2, r3, F):
    return r1 + F * (r2 - r3)

def crossover(target, mutant, CR):
    trial = target.copy()
    dim = len(target)
    j_rand = np.random.randint(dim)
    for j in range(dim):
        if np.random.rand() < CR or j == j_rand:
            trial[j] = mutant[j]
    return trial

class ABC_SMC_DE:
    def __init__(self, N, eps_schedule, F=0.5, CR=0.9, strategy='rand/1', max_trials=100000):
        self.N = N
        self.eps_schedule = eps_schedule
        self.F = F
        self.CR = CR
        self.strategy = strategy
        self.max_trials = max_trials

    
    def run(self):
        particles, histories = [], []

        for t, eps in enumerate(self.eps_schedule):
            print(f"\n Population {t} (epsilon={eps:.3f})")
            prev_p = particles[-1] if t > 0 else None
            prev_s = histories[-1] if t > 0 else None

            accepted, sims = [], []
            trials = 0
            oversample = 5

            while len(accepted) < self.N and trials < self.max_trials:
                n_candidates = self.N * oversample
                proposals = []
                for _ in range(n_candidates):
                    theta = sample_prior() if prev_p is None else self.propose(theta_count=len(prev_p), prev_p=prev_p, prev_s=prev_s)
                    proposals.append(theta)

                # results = Parallel(n_jobs=-1)(

                results = Parallel(n_jobs=7)(

                    delayed(simulate)(theta) for theta in proposals

                )
                trials += len(results)

                for theta, sim_rmse in zip(proposals, results):
                    if sim_rmse <= eps:
                        accepted.append(theta)
                        sims.append(sim_rmse)
                        print(f"  {len(accepted)}. lr={theta[0]:.4f}, depth={int(round(theta[1]))}, reg={theta[2]:.4f}, RMSE={sim_rmse:.4f}")
                        if len(accepted) >= self.N:
                            break

                if len(accepted) == 0:
                    print(f"No acceptances after {trials} trials")
                    break

            if not accepted:
                raise RuntimeError(f"Population {t} failed: no accepts.")

            particles.append(np.vstack(accepted))
            histories.append(sims)
            print(f"Accepted {len(accepted)} in {trials} trials.")

        return particles, histories

    def propose(self, theta_count, prev_p, prev_s):
        N = theta_count
        if self.strategy == 'rand/1':
            i1, i2, i3 = np.random.choice(N, 3, replace=False)
            mutant = de_mutate(prev_p[i1], prev_p[i2], prev_p[i3], self.F)
        else:  # best/1
            best_idx = np.argmin(prev_s)
            r2, r3 = np.random.choice(N, 2, replace=False)
            mutant = de_mutate(prev_p[best_idx], prev_p[r2], prev_p[r3], self.F)

        target_idx = np.random.randint(N)
        theta = crossover(prev_p[target_idx], mutant, self.CR)
        theta[0] = np.clip(theta[0], 1e-4, 1.0)
        theta[1] = np.clip(theta[1], 2, 16)
        theta[2] = max(theta[2], 1e-6)
        return theta

if __name__ == '__main__':
    # print(eps_schedule)
    abc_de = ABC_SMC_DE(
        N=100,
        eps_schedule=eps_schedule,
        F=0.6,
        CR=0.9,
        strategy='best/1',
        max_trials=200000
    )
    particles, histories = abc_de.run()

    final_s = histories[-1]
    best_idx = np.argmin(final_s)
    best_params = particles[-1][best_idx]
    best_lr, best_depth, best_reg = best_params

    print("\nBest hyperparameters:")
    print(f"  lr={best_lr:.4f}, depth={int(round(best_depth))}, reg={best_reg:.4f}")

    best_model = CatBoostRegressor(
        learning_rate=best_lr,
        depth=int(round(best_depth)),
        l2_leaf_reg=best_reg,
        iterations=500,
        random_seed=0,
        verbose=False
    )
    best_model.fit(Pool(X_train_full, y_train_full))
    preds = best_model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, preds))
    r2_test = r2_score(y_test, preds)
    print(f"Final Test RMSE: {rmse_test:.4f}, R²: {r2_test:.4f}")
    param_names = ['learning_rate', 'depth', 'l2_leaf_reg']
    for i, name in enumerate(param_names):
        plt.figure(figsize=(6, 2))
        for t, pop in enumerate(particles):
            sns.kdeplot(pop[:, i], label=f"Gen {t}")
        plt.title(f"Posterior evolution for {name}")
        plt.xlabel(name)
        plt.ylabel("Density")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    median_rmses = [np.median(s) for s in histories]
    min_rmses = [np.min(s) for s in histories]
    plt.figure(figsize=(7, 2))
    plt.plot(median_rmses, label="Median RMSE")
    plt.plot(min_rmses, label="Min RMSE")
    plt.xlabel("Population")
    plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
    plt.xticks(range(0,4))
    plt.ylabel("Validation RMSE")
    plt.title("ABC-SMC-DE: RMSE over Populations")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

Baseline RMSE: 48.4266

 Population 0 (epsilon=50.848)
  1. lr=0.0941, depth=10, reg=2.3862, RMSE=48.5709
  2. lr=0.0985, depth=4, reg=3.7819, RMSE=50.1271
  3. lr=0.1366, depth=5, reg=4.3804, RMSE=49.6504
  4. lr=0.1331, depth=4, reg=2.5622, RMSE=49.2110
  5. lr=0.0444, depth=5, reg=0.7752, RMSE=50.2279
  6. lr=0.0542, depth=7, reg=2.0841, RMSE=49.6499
  7. lr=0.0835, depth=6, reg=3.1516, RMSE=50.0668
  8. lr=0.0935, depth=4, reg=0.8725, RMSE=48.1800
  9. lr=0.0943, depth=9, reg=0.2081, RMSE=49.3256
  10. lr=0.1006, depth=4, reg=3.5464, RMSE=49.9724
  11. lr=0.1451, depth=6, reg=3.3773, RMSE=47.5179
  12. lr=0.0501, depth=6, reg=0.7693, RMSE=49.9837
  13. lr=0.0937, depth=4, reg=3.5584, RMSE=49.4978
  14. lr=0.1062, depth=6, reg=2.8219, RMSE=48.6221
  15. lr=0.0567, depth=8, reg=0.6829, RMSE=49.8062
  16. lr=0.1465, depth=3, reg=1.1296, RMSE=50.1988
  17. lr=0.1228, depth=4, reg=4.5274, RMSE=50.6405
  18. lr=0.1158, depth=7, reg=1.4151, RMSE=48.1979
  19. lr=0.1425, depth=3, reg=4.698

RuntimeError: Population 9 failed: no accepts.

In [2]:
##### hybrid
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool
from joblib import Parallel, delayed
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Load data
X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv').squeeze()

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_full, y_train_full, test_size=0.3, random_state=1)

baseline = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    random_seed=0,
    verbose=False,
    early_stopping_rounds=50
)
baseline.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
baseline_rmse = np.sqrt(mean_squared_error(y_val, baseline.predict(X_val)))
print(f"Baseline RMSE: {baseline_rmse:.4f}")

eps_schedule = [baseline_rmse * f for f in (1.05, 1.0, 0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.90)]

def sample_prior():
    return np.array([
        np.random.uniform(0.03, 0.15),  # learning_rate
        np.random.uniform(3, 10),       # depth
        np.random.uniform(0.01, 5)      # l2_leaf_reg
    ])

def simulate(theta):
    lr, depth, reg = theta
    model = CatBoostRegressor(
        learning_rate=lr,
        depth=int(round(depth)),
        l2_leaf_reg=reg,
        iterations=50,
        early_stopping_rounds=10,
        random_seed=0,
        verbose=False
    )
    model.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

def de_mutate(r1, r2, r3, F):
    return r1 + F * (r2 - r3)

def crossover(target, mutant, CR):
    trial = target.copy()
    dim = len(target)
    j_rand = np.random.randint(dim)
    for j in range(dim):
        if np.random.rand() < CR or j == j_rand:
            trial[j] = mutant[j]
    return trial

class ABC_SMC_DE:
    def __init__(self, N, eps_schedule, F=0.5, CR=0.9, max_trials=100000):
        self.N = N
        self.eps_schedule = eps_schedule
        self.F = F
        self.CR = CR
        self.max_trials = max_trials

    def compute_diversity(self, population):
        return np.mean(np.std(population, axis=0))

    def propose(self, prev_p, prev_s, diversity_threshold=0.1):
        N = len(prev_p)
        diversity = self.compute_diversity(prev_p)

        use_jitter = np.random.rand() < 0.2  # exploration
        if use_jitter:
            idx = np.random.randint(N)
            theta = prev_p[idx] + np.random.normal(0, [0.01, 0.5, 0.1], size=3)
        else:
            if diversity > diversity_threshold:
                # rand/1
                i1, i2, i3 = np.random.choice(N, 3, replace=False)
                mutant = de_mutate(prev_p[i1], prev_p[i2], prev_p[i3], self.F)
            else:
                # best/1
                best_idx = np.argmin(prev_s)
                r2, r3 = np.random.choice(N, 2, replace=False)
                mutant = de_mutate(prev_p[best_idx], prev_p[r2], prev_p[r3], self.F)

            target_idx = np.random.randint(N)
            theta = crossover(prev_p[target_idx], mutant, self.CR)

        theta[0] = np.clip(theta[0], 0.01, 0.2)
        theta[1] = np.clip(theta[1], 3, 10)
        theta[2] = np.clip(theta[2], 0.01, 5)
        return theta

    def run(self):
        particles, histories = [], []

        for t, eps in enumerate(self.eps_schedule):
            print(f"\n Population {t} (epsilon={eps:.3f})")
            prev_p = particles[-1] if t > 0 else None
            prev_s = histories[-1] if t > 0 else None

            accepted, sims = [], []
            trials = 0
            oversample = 5

            while len(accepted) < self.N and trials < self.max_trials:
                n_candidates = self.N * oversample
                proposals = []
                for _ in range(n_candidates):
                    if prev_p is None:
                        theta = sample_prior()
                    else:
                        theta = self.propose(prev_p, prev_s)
                    proposals.append(theta)

                results = Parallel(n_jobs=7)(
                    delayed(simulate)(theta) for theta in proposals
                )
                trials += len(results)

                for theta, sim_rmse in zip(proposals, results):
                    if sim_rmse <= eps:
                        accepted.append(theta)
                        sims.append(sim_rmse)
                        print(f"  {len(accepted)}. lr={theta[0]:.4f}, depth={int(round(theta[1]))}, reg={theta[2]:.4f}, RMSE={sim_rmse:.4f}")
                        if len(accepted) >= self.N:
                            break

                if len(accepted) == 0:
                    print(f"No acceptances after {trials} trials")
                    break

            if not accepted:
                raise RuntimeError(f"Population {t} failed: no accepts.")

            particles.append(np.vstack(accepted))
            histories.append(sims)
            print(f"Accepted {len(accepted)} in {trials} trials.")

        return particles, histories

if __name__ == '__main__':
    abc_de = ABC_SMC_DE(
        N=100,
        eps_schedule=eps_schedule,
        F=0.6,
        CR=0.9,
        max_trials=200000
    )
    particles, histories = abc_de.run()

    final_s = histories[-1]
    best_idx = np.argmin(final_s)
    best_params = particles[-1][best_idx]
    best_lr, best_depth, best_reg = best_params

    print("\nBest hyperparameters:")
    print(f"  lr={best_lr:.4f}, depth={int(round(best_depth))}, reg={best_reg:.4f}")

    best_model = CatBoostRegressor(
        learning_rate=best_lr,
        depth=int(round(best_depth)),
        l2_leaf_reg=best_reg,
        iterations=500,
        random_seed=0,
        verbose=False
    )
    best_model.fit(Pool(X_train_full, y_train_full))
    preds = best_model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, preds))
    r2_test = r2_score(y_test, preds)
    print(f"Final Test RMSE: {rmse_test:.4f}, R²: {r2_test:.4f}")

Baseline RMSE: 48.4266

 Population 0 (epsilon=50.848)
  1. lr=0.1091, depth=5, reg=0.2894, RMSE=49.6443
  2. lr=0.1179, depth=4, reg=1.7178, RMSE=50.3430
  3. lr=0.0783, depth=9, reg=3.0297, RMSE=50.7428
  4. lr=0.1414, depth=10, reg=2.2016, RMSE=47.2232
  5. lr=0.1267, depth=7, reg=2.1718, RMSE=48.4231
  6. lr=0.0630, depth=8, reg=2.4432, RMSE=50.5185
  7. lr=0.1286, depth=6, reg=2.4285, RMSE=48.5798
  8. lr=0.1093, depth=8, reg=0.7878, RMSE=47.3614
  9. lr=0.0843, depth=8, reg=3.7839, RMSE=50.2411
  10. lr=0.1065, depth=10, reg=0.7795, RMSE=46.3920
  11. lr=0.0635, depth=5, reg=1.3025, RMSE=48.9506
  12. lr=0.0693, depth=7, reg=2.6032, RMSE=49.0503
  13. lr=0.0689, depth=6, reg=0.0414, RMSE=49.8143
  14. lr=0.1264, depth=6, reg=4.7942, RMSE=49.1320
  15. lr=0.0607, depth=5, reg=4.5240, RMSE=50.0400
  16. lr=0.0933, depth=4, reg=0.4874, RMSE=49.6940
  17. lr=0.1498, depth=10, reg=0.6590, RMSE=47.6890
  18. lr=0.0824, depth=6, reg=0.6485, RMSE=47.4791
  19. lr=0.0765, depth=9, reg=0.9

RuntimeError: Population 6 failed: no accepts.

In [3]:
#######  current-to-best/1
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool
from joblib import Parallel, delayed
import seaborn as sns
import matplotlib.pyplot as plt

X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv').squeeze()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.3, random_state=1
)

# get RMSE scale
baseline = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    random_seed=0,
    verbose=False,
    early_stopping_rounds=50
)
baseline.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
baseline_rmse = np.sqrt(mean_squared_error(y_val, baseline.predict(X_val)))
print(f"Baseline RMSE: {baseline_rmse:.4f}")

eps_schedule = [baseline_rmse * f for f in (1.05, 1.0, 0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.90, 0.89, 0.88, 0.87, 0.86, 0.85)]

def sample_prior():
    return np.array([
        np.random.uniform(0.03, 0.15),  # learning_rate
        np.random.uniform(3, 10),       # depth
        np.random.uniform(0.01, 5)      # l2_leaf_reg
    ])

def simulate(theta):
    lr, depth, reg = theta
    model = CatBoostRegressor(
        learning_rate=lr,
        depth=int(round(depth)),
        l2_leaf_reg=reg,
        iterations=50,
        early_stopping_rounds=10,
        random_seed=0,
        verbose=False
    )
    model.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

def de_mutate(r1, r2, r3, F):
    return r1 + F * (r2 - r3)

def crossover(target, mutant, CR):
    trial = target.copy()
    dim = len(target)
    j_rand = np.random.randint(dim)
    for j in range(dim):
        if np.random.rand() < CR or j == j_rand:
            trial[j] = mutant[j]
    return trial

class ABC_SMC_DE:
    def __init__(self, N, eps_schedule, F=0.5, CR=0.9, strategy='rand/1', max_trials=100000):
        self.N = N
        self.eps_schedule = eps_schedule
        self.F = F
        self.CR = CR
        self.strategy = strategy
        self.max_trials = max_trials

    
    def run(self):
        particles, histories = [], []

        for t, eps in enumerate(self.eps_schedule):
            print(f"\n Population {t} (epsilon={eps:.3f})")
            prev_p = particles[-1] if t > 0 else None
            prev_s = histories[-1] if t > 0 else None

            accepted, sims = [], []
            trials = 0
            oversample = 5

            while len(accepted) < self.N and trials < self.max_trials:
                n_candidates = self.N * oversample
                proposals = []
                for _ in range(n_candidates):
                    theta = sample_prior() if prev_p is None else self.propose(theta_count=len(prev_p), prev_p=prev_p, prev_s=prev_s)
                    proposals.append(theta)

                # results = Parallel(n_jobs=-1)(

                results = Parallel(n_jobs=7)(

                    delayed(simulate)(theta) for theta in proposals

                )
                trials += len(results)

                for theta, sim_rmse in zip(proposals, results):
                    if sim_rmse <= eps:
                        accepted.append(theta)
                        sims.append(sim_rmse)
                        print(f"  {len(accepted)}. lr={theta[0]:.4f}, depth={int(round(theta[1]))}, reg={theta[2]:.4f}, RMSE={sim_rmse:.4f}")
                        if len(accepted) >= self.N:
                            break

                if len(accepted) == 0:
                    print(f"No acceptances after {trials} trials")
                    break

            if not accepted:
                raise RuntimeError(f"Population {t} failed: no accepts.")

            particles.append(np.vstack(accepted))
            histories.append(sims)
            print(f"Accepted {len(accepted)} in {trials} trials.")

        return particles, histories

    def propose(self, theta_count, prev_p, prev_s):
        N = theta_count
        best_idx = np.argmin(prev_s)
        best = prev_p[best_idx]

        if self.strategy == 'current-to-best/1':
            i1, r2, r3 = np.random.choice(N, 3, replace=False)
            current = prev_p[i1]
            mutant = current + self.F * (best - current + prev_p[r2] - prev_p[r3])

        elif self.strategy == 'rand-to-best/1':
            i1, r2, r3 = np.random.choice(N, 3, replace=False)
            r1 = prev_p[i1]
            mutant = r1 + self.F * (best - r1) + self.F * (prev_p[r2] - prev_p[r3])

        else:
            raise ValueError(f"Unknown DE strategy: {self.strategy}")

        target_idx = np.random.randint(N)
        theta = crossover(prev_p[target_idx], mutant, self.CR)

        theta[0] = np.clip(theta[0], 0.01, 0.2)
        theta[1] = np.clip(theta[1], 3, 10)
        theta[2] = np.clip(theta[2], 0.01, 5)
        return theta


if __name__ == '__main__':
    # print(eps_schedule)
    abc_de = ABC_SMC_DE(
        N=100,
        eps_schedule=eps_schedule,
        F=0.6,
        CR=0.9,
        strategy='current-to-best/1',
        max_trials=200000
    )
    particles, histories = abc_de.run()

    final_s = histories[-1]
    best_idx = np.argmin(final_s)
    best_params = particles[-1][best_idx]
    best_lr, best_depth, best_reg = best_params

    print("\nBest hyperparameters:")
    print(f"  lr={best_lr:.4f}, depth={int(round(best_depth))}, reg={best_reg:.4f}")

    best_model = CatBoostRegressor(
        learning_rate=best_lr,
        depth=int(round(best_depth)),
        l2_leaf_reg=best_reg,
        iterations=500,
        random_seed=0,
        verbose=False
    )
    best_model.fit(Pool(X_train_full, y_train_full))
    preds = best_model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, preds))
    r2_test = r2_score(y_test, preds)
    print(f"Final Test RMSE: {rmse_test:.4f}, R²: {r2_test:.4f}")
    param_names = ['learning_rate', 'depth', 'l2_leaf_reg']
    for i, name in enumerate(param_names):
        plt.figure(figsize=(6, 2))
        for t, pop in enumerate(particles):
            sns.kdeplot(pop[:, i], label=f"Gen {t}")
        plt.title(f"Posterior evolution for {name}")
        plt.xlabel(name)
        plt.ylabel("Density")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    median_rmses = [np.median(s) for s in histories]
    min_rmses = [np.min(s) for s in histories]
    plt.figure(figsize=(7, 2))
    plt.plot(median_rmses, label="Median RMSE")
    plt.plot(min_rmses, label="Min RMSE")
    plt.xlabel("Population")
    plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
    plt.xticks(range(0,4))
    plt.ylabel("Validation RMSE")
    plt.title("ABC-SMC-DE: RMSE over Populations")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

Baseline RMSE: 48.4266

 Population 0 (epsilon=50.848)
  1. lr=0.1372, depth=5, reg=3.1827, RMSE=50.1796
  2. lr=0.0965, depth=6, reg=1.8552, RMSE=49.5869
  3. lr=0.1427, depth=6, reg=1.1046, RMSE=50.1978
  4. lr=0.1199, depth=7, reg=3.8022, RMSE=49.4864
  5. lr=0.0776, depth=10, reg=1.5375, RMSE=50.7137
  6. lr=0.0944, depth=9, reg=0.6780, RMSE=50.5079
  7. lr=0.1018, depth=6, reg=3.0502, RMSE=49.3048
  8. lr=0.1373, depth=5, reg=2.9782, RMSE=49.8246
  9. lr=0.1025, depth=9, reg=2.4451, RMSE=50.3635
  10. lr=0.1030, depth=10, reg=0.8148, RMSE=49.7525
  11. lr=0.0781, depth=4, reg=3.7903, RMSE=48.9706
  12. lr=0.0836, depth=4, reg=1.0749, RMSE=49.8026
  13. lr=0.1345, depth=5, reg=2.5830, RMSE=50.5662
  14. lr=0.0747, depth=7, reg=2.5225, RMSE=48.1166
  15. lr=0.1432, depth=7, reg=4.9909, RMSE=49.7804
  16. lr=0.0913, depth=8, reg=2.7662, RMSE=49.4691
  17. lr=0.1067, depth=4, reg=1.5065, RMSE=49.9157
  18. lr=0.0957, depth=3, reg=3.6348, RMSE=49.6024
  19. lr=0.0847, depth=3, reg=0.18

RuntimeError: Population 7 failed: no accepts.

In [4]:
#######  rand-to-best/1
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from catboost import CatBoostRegressor, Pool
from joblib import Parallel, delayed
import seaborn as sns
import matplotlib.pyplot as plt

X = pd.read_csv('X.csv')
y = pd.read_csv('y.csv').squeeze()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_full, y_train_full, test_size=0.3, random_state=1
)

# get RMSE scale
baseline = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    random_seed=0,
    verbose=False,
    early_stopping_rounds=50
)
baseline.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
baseline_rmse = np.sqrt(mean_squared_error(y_val, baseline.predict(X_val)))
print(f"Baseline RMSE: {baseline_rmse:.4f}")

eps_schedule = [baseline_rmse * f for f in (1.05, 1.0, 0.99, 0.98, 0.97, 0.96, 0.95, 0.94, 0.93, 0.92, 0.91, 0.90, 0.89, 0.88, 0.87, 0.86, 0.85)]

def sample_prior():
    return np.array([
        np.random.uniform(0.03, 0.15),  # learning_rate
        np.random.uniform(3, 10),       # depth
        np.random.uniform(0.01, 5)      # l2_leaf_reg
    ])

def simulate(theta):
    lr, depth, reg = theta
    model = CatBoostRegressor(
        learning_rate=lr,
        depth=int(round(depth)),
        l2_leaf_reg=reg,
        iterations=50,
        early_stopping_rounds=10,
        random_seed=0,
        verbose=False
    )
    model.fit(Pool(X_train, y_train), eval_set=Pool(X_val, y_val))
    preds = model.predict(X_val)
    return np.sqrt(mean_squared_error(y_val, preds))

def de_mutate(r1, r2, r3, F):
    return r1 + F * (r2 - r3)

def crossover(target, mutant, CR):
    trial = target.copy()
    dim = len(target)
    j_rand = np.random.randint(dim)
    for j in range(dim):
        if np.random.rand() < CR or j == j_rand:
            trial[j] = mutant[j]
    return trial

class ABC_SMC_DE:
    def __init__(self, N, eps_schedule, F=0.5, CR=0.9, strategy='rand/1', max_trials=100000):
        self.N = N
        self.eps_schedule = eps_schedule
        self.F = F
        self.CR = CR
        self.strategy = strategy
        self.max_trials = max_trials

    
    def run(self):
        particles, histories = [], []

        for t, eps in enumerate(self.eps_schedule):
            print(f"\n Population {t} (epsilon={eps:.3f})")
            prev_p = particles[-1] if t > 0 else None
            prev_s = histories[-1] if t > 0 else None

            accepted, sims = [], []
            trials = 0
            oversample = 5

            while len(accepted) < self.N and trials < self.max_trials:
                n_candidates = self.N * oversample
                proposals = []
                for _ in range(n_candidates):
                    theta = sample_prior() if prev_p is None else self.propose(theta_count=len(prev_p), prev_p=prev_p, prev_s=prev_s)
                    proposals.append(theta)

                # results = Parallel(n_jobs=-1)(

                results = Parallel(n_jobs=7)(

                    delayed(simulate)(theta) for theta in proposals

                )
                trials += len(results)

                for theta, sim_rmse in zip(proposals, results):
                    if sim_rmse <= eps:
                        accepted.append(theta)
                        sims.append(sim_rmse)
                        print(f"  {len(accepted)}. lr={theta[0]:.4f}, depth={int(round(theta[1]))}, reg={theta[2]:.4f}, RMSE={sim_rmse:.4f}")
                        if len(accepted) >= self.N:
                            break

                if len(accepted) == 0:
                    print(f"No acceptances after {trials} trials")
                    break

            if not accepted:
                raise RuntimeError(f"Population {t} failed: no accepts.")

            particles.append(np.vstack(accepted))
            histories.append(sims)
            print(f"Accepted {len(accepted)} in {trials} trials.")

        return particles, histories

    def propose(self, theta_count, prev_p, prev_s):
        N = theta_count
        best_idx = np.argmin(prev_s)
        best = prev_p[best_idx]

        if self.strategy == 'current-to-best/1':
            i1, r2, r3 = np.random.choice(N, 3, replace=False)
            current = prev_p[i1]
            mutant = current + self.F * (best - current + prev_p[r2] - prev_p[r3])

        elif self.strategy == 'rand-to-best/1':
            i1, r2, r3 = np.random.choice(N, 3, replace=False)
            r1 = prev_p[i1]
            mutant = r1 + self.F * (best - r1) + self.F * (prev_p[r2] - prev_p[r3])

        else:
            raise ValueError(f"Unknown DE strategy: {self.strategy}")

        target_idx = np.random.randint(N)
        theta = crossover(prev_p[target_idx], mutant, self.CR)

        theta[0] = np.clip(theta[0], 0.01, 0.2)
        theta[1] = np.clip(theta[1], 3, 10)
        theta[2] = np.clip(theta[2], 0.01, 5)
        return theta


if __name__ == '__main__':
    # print(eps_schedule)
    abc_de = ABC_SMC_DE(
        N=100,
        eps_schedule=eps_schedule,
        F=0.6,
        CR=0.9,
        strategy='rand-to-best/1',
        max_trials=200000
    )
    particles, histories = abc_de.run()

    final_s = histories[-1]
    best_idx = np.argmin(final_s)
    best_params = particles[-1][best_idx]
    best_lr, best_depth, best_reg = best_params

    print("\nBest hyperparameters:")
    print(f"  lr={best_lr:.4f}, depth={int(round(best_depth))}, reg={best_reg:.4f}")

    best_model = CatBoostRegressor(
        learning_rate=best_lr,
        depth=int(round(best_depth)),
        l2_leaf_reg=best_reg,
        iterations=500,
        random_seed=0,
        verbose=False
    )
    best_model.fit(Pool(X_train_full, y_train_full))
    preds = best_model.predict(X_test)
    rmse_test = np.sqrt(mean_squared_error(y_test, preds))
    r2_test = r2_score(y_test, preds)
    print(f"Final Test RMSE: {rmse_test:.4f}, R²: {r2_test:.4f}")
    param_names = ['learning_rate', 'depth', 'l2_leaf_reg']
    for i, name in enumerate(param_names):
        plt.figure(figsize=(6, 2))
        for t, pop in enumerate(particles):
            sns.kdeplot(pop[:, i], label=f"Gen {t}")
        plt.title(f"Posterior evolution for {name}")
        plt.xlabel(name)
        plt.ylabel("Density")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()
        plt.show()

    median_rmses = [np.median(s) for s in histories]
    min_rmses = [np.min(s) for s in histories]
    plt.figure(figsize=(7, 2))
    plt.plot(median_rmses, label="Median RMSE")
    plt.plot(min_rmses, label="Min RMSE")
    plt.xlabel("Population")
    plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
    plt.xticks(range(0,4))
    plt.ylabel("Validation RMSE")
    plt.title("ABC-SMC-DE: RMSE over Populations")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

Baseline RMSE: 48.4266

 Population 0 (epsilon=50.848)
  1. lr=0.1312, depth=8, reg=2.1545, RMSE=49.6484
  2. lr=0.0917, depth=6, reg=2.6440, RMSE=49.7576
  3. lr=0.1118, depth=10, reg=2.7505, RMSE=49.0156
  4. lr=0.1012, depth=3, reg=3.0594, RMSE=49.6273
  5. lr=0.1274, depth=9, reg=2.5993, RMSE=50.3006
  6. lr=0.1317, depth=7, reg=3.6544, RMSE=50.0704
  7. lr=0.1043, depth=8, reg=2.0581, RMSE=49.8505
  8. lr=0.0817, depth=5, reg=0.2965, RMSE=48.2846
  9. lr=0.1362, depth=4, reg=0.5726, RMSE=49.5540
  10. lr=0.1366, depth=5, reg=3.9856, RMSE=50.0806
  11. lr=0.1177, depth=7, reg=4.7354, RMSE=49.0964
  12. lr=0.1283, depth=4, reg=4.1349, RMSE=50.2990
  13. lr=0.1217, depth=9, reg=1.9773, RMSE=50.7459
  14. lr=0.0813, depth=7, reg=3.0440, RMSE=49.6555
  15. lr=0.1285, depth=6, reg=4.7878, RMSE=49.1138
  16. lr=0.1121, depth=3, reg=3.3412, RMSE=49.7597
  17. lr=0.0407, depth=5, reg=0.2502, RMSE=50.3997
  18. lr=0.0536, depth=9, reg=1.1514, RMSE=50.5775
  19. lr=0.1497, depth=6, reg=1.407

RuntimeError: Population 8 failed: no accepts.